In [27]:
#H2o R

In [48]:
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/RtmpFPxHqb/h2o_rbhat_started_from_r.out
    /tmp/RtmpFPxHqb/h2o_rbhat_started_from_r.err


Starting H2O JVM and connecting: .. Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 seconds 685 milliseconds 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_xtu945 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.60 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 

Note:  As started, H2O is limited to the CRAN default of 2 CPUs.
       Shut down and restart H2O as shown below to use all your CPUs.
           > h2o.s

In [12]:


## Import MNIST CSV as H2O:
mnistPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/train.csv"
mnist.hex = h2o.importFile(path = mnistPath, destination_frame = "mnist.hex")
train <- as.data.frame(mnist.hex)
train$label <- as.factor(train$label)
train_h2o <- as.h2o(train)

#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning(x = 2:785, # column numbers for predictors
                   y = 1,            # column number for label
                   training_frame = train_h2o,
                   activation = "RectifierWithDropout", 
                   input_dropout_ratio = 0.2,
                   hidden_dropout_ratios = c(0.5,0.5),
                   balance_classes = TRUE, 
                   hidden = c(800,800),  ## five hidden layers
                   epochs = 10,
                   variable_importances = T)

#Predict on test data:
test_h2o <- h2o.importFile(path = '/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/test.csv', 
                           destination_frame = "test_h2o")
yhat <- h2o.predict(model, test_h2o)
ImageId <- as.numeric(seq(1,28000))
names(ImageId)[1] <- "ImageId"
predictions <- cbind(as.data.frame(ImageId),as.data.frame(yhat[,1]))
names(predictions)[2] <- "Label"
write.table(as.matrix(predictions), file="/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/DNN_pred.csv", 
            row.names=FALSE, sep=",")

  |======================================================================| 100%
  |======================================================================| 100%


Warning message:
In .h2o.validateModelParameters(algo, param_values, h2oRestApiVersion): Dropping constant columns: [pixel673, pixel196, pixel672, pixel671, pixel57, pixel476, pixel10, pixel532, pixel11, pixel16, pixel52, pixel730, pixel53, pixel18, pixel731, pixel54, pixel17, pixel392, pixel55, pixel56, pixel19, pixel782, pixel2, pixel781, pixel3, pixel780, pixel4, pixel5, pixel448, pixel0, pixel783, pixel168, pixel1, pixel700, pixel701, pixel645, pixel644, pixel112, pixel111, pixel7, pixel6, pixel9, pixel8, pixel759, pixel758, pixel757, pixel756, pixel755, pixel754, pixel31, pixel30, pixel699, pixel139, pixel729, pixel141, pixel140, pixel727, pixel728, pixel23, pixel22, pixel21, pixel20, pixel560, pixel29, pixel28, pixel82, pixel27, pixel83, pixel26, pixel421, pixel84, pixel25, pixel760, pixel420, pixel85, pixel24].


  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


In [13]:
h2o.varimp(model)

,variable,relative_importance,scaled_importance,percentage
1,pixel768,1,1,0.00219971489856623
2,pixel39,0.99789834022522,0.99789834022522,0.00219509184624793
3,pixel769,0.985796093940735,0.985796093940735,0.00216847035478983
4,pixel772,0.974150478839874,0.974150478839874,0.0021428533217495
5,pixel586,0.97146075963974,0.97146075963974,0.00213693670635201
6,pixel765,0.970835983753204,0.970835983753204,0.00213556237752613
7,pixel763,0.969893515110016,0.969893515110016,0.00213348921521028
8,pixel36,0.969248294830322,0.969248294830322,0.00213206991454818
9,pixel47,0.956517696380615,0.956517696380615,0.00210406622747069
10,pixel558,0.955120742321014,0.955120742321014,0.00210099332681317


In [19]:
##########
## Data Source References:
## MNIST Source Data
## Original Dataset: http://yann.lecun.com/exdb/mnist/
## CSV Version: http://www.pjreddie.com/projects/mnist-in-csv/
## Train Dataset: http://www.pjreddie.com/media/files/mnist_train.csv
## Test Dataset: http://www.pjreddie.com/media/files/mnist_test.csv

#https://github.com/woobe/blenditbayes/blob/master/2014-07-h2o-experiments/code_h2o_dnn_exp_2.R

In [18]:
## Start a local H2O cluster directly from R
# localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE)  # default settings with 1GB RAM
#localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE, Xmx = '2g') # cap it at max 2GB